---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [32]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [33]:
state = None
data = []
with open('university_towns.txt') as file:
    for line in file:
        line = line[:-1]
        if line.endswith('[edit]'):
            state = line[:-6]
        else:
            if '(' in line:
                data.append((state, line[:line.index('(') - 1]))
            else:
                data.append((state, line))
uni_town = pd.DataFrame(data, columns=['State', 'RegionName'])
uni_town.head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [34]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [35]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return uni_town

In [36]:
gdp = pd.DataFrame([[np.nan, 0] for i in range(2)], columns=['quarter', 'dollar'])
temp = pd.read_excel('gdplev.xls', skiprows=220, usecols=[4, 6], header=None, columns=['quarter', 'dollar'])
temp.columns = ['quarter', 'dollar']
gdp = gdp.append(temp, ignore_index=True)
gdp.head()

,quarter,dollar
0,NaN,0.0
1,NaN,0.0
2,2000q1,12359.1
3,2000q2,12592.5
4,2000q3,12607.7


In [37]:
gdp.iloc[range(34, 50)]

,quarter,dollar
34,2008q1,14889.5
35,2008q2,14963.4
36,2008q3,14891.6
37,2008q4,14577.0
38,2009q1,14375.0
39,2009q2,14355.6
40,2009q3,14402.5
41,2009q4,14541.9
42,2010q1,14604.8
43,2010q2,14745.9


In [38]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    dollar = gdp['dollar']
    for i in range(2, len(dollar)):
        if dollar[i - 2] > dollar[i - 1] and dollar[i - 1] > dollar[i]:
            return gdp.iloc[i - 1]['quarter']

In [39]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    start_index = gdp[gdp['quarter'] == get_recession_start()].index[0]
    dollar = gdp['dollar']
    for i in range(start_index, len(dollar)):
        if dollar[i - 2] < dollar[i - 1] and dollar[i - 1] < dollar[i]:
            return gdp.iloc[i]['quarter']

In [40]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    start_index = gdp[gdp['quarter'] == get_recession_start()].index[0]
    end_index = gdp[gdp['quarter'] == get_recession_end()].index[0]
    recession_time = gdp.iloc[range(start_index, end_index + 1)]
    bottom_index = recession_time['dollar'].nsmallest(1).index[0]
    return gdp.iloc[bottom_index]['quarter']

In [41]:
houses = pd.read_csv('City_Zhvi_AllHomes.csv')
houses = houses.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1)
houses = houses.drop([col for col in  houses.columns[2:47]], axis=1)
houses.head()

,RegionName,State,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,Los Angeles,CA,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,Chicago,IL,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,Philadelphia,PA,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,Phoenix,AZ,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [42]:
quarter_houses = houses[['RegionName', 'State']]
quarter_houses.head()

,RegionName,State
0,New York,NY
1,Los Angeles,CA
2,Chicago,IL
3,Philadelphia,PA
4,Phoenix,AZ


In [43]:
quarter_dict = {'q1': ['01', '02', '03'], 'q2': ['04', '05', '06'], 'q3': ['07', '08', '09'], 'q4': ['10', '11', '12']}

In [44]:
for year in range(2000, 2016):
    for quarter in quarter_dict:
        quarter_houses[str(year) + quarter] = houses[[str(year) + '-' + month for month in quarter_dict[quarter]]].mean(axis=1)
quarter_houses['2016q1'] = houses[[str(2016) + '-' + month for month in quarter_dict['q1']]].mean(axis=1)
quarter_houses['2016q2'] = houses[[str(2016) + '-' + month for month in quarter_dict['q2']]].mean(axis=1)
quarter_houses['2016q3'] = houses[['2016-07', '2016-08']].mean(axis=1)
quarter_houses = quarter_houses.replace({'State':states})
quarter_houses = quarter_houses.set_index(['State', 'RegionName'])
# quarter_houses

In [45]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return quarter_houses

In [59]:
convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [47]:
uni_towns = get_list_of_university_towns()
recession_start = get_recession_start()
recession_end = get_recession_end()
recession_bottom = get_recession_bottom()
hdata = convert_housing_data_to_quarters().copy()

In [48]:
bstart = hdata.columns[hdata.columns.get_loc(recession_start) -1]
hdata['ratio'] = hdata[bstart] / hdata[recession_bottom]

In [49]:
uni_town_data = hdata.merge(uni_towns, left_index=True, right_on=['State', 'RegionName'])
uni_town_data = uni_town_data.set_index(['State', 'RegionName'])
uni_town_data

2000q1         2000q2         2000q3  \
State          RegionName                                                      
Nevada         Las Vegas         132600.000000  134366.666667  135400.000000   
California     San Diego         222900.000000  234366.666667  245433.333333   
Texas          Dallas             84466.666667   83866.666667   84866.666667   
               Austin            142966.666667  145266.666667  149466.666667   
Ohio           Columbus           94366.666667   95833.333333   97133.333333   
Tennessee      Memphis            72500.000000   73200.000000   73866.666667   
Massachusetts  Boston            206933.333333  219166.666667  233100.000000   
Tennessee      Nashville         113833.333333  115266.666667  115866.666667   
Wisconsin      Milwaukee          78033.333333   79066.666667   81033.333333   
Arizona        Tucson            101833.333333  102966.666667  104466.666667   
California     Sacramento        122433.333333  126000.000000  129466.666667   
Georgia        Atlanta           143566.666667  147966.666667  151266.666667   
Oklahoma       Tulsa              73866.666667   75166.666667   76300.000000   
Florida        Tampa              79100.000000   81433.333333   82800.000000   
California     Riverside         137166.666667  139666.666667  141300.000000   
Kentucky       Lexington         113033.333333  115166.666667  116366.666667   
New Jersey     Newark            124466.666667  127700.000000  130433.333333   
Florida        Tallahassee       103100.000000  104366.666667  105000.000000   
Kentucky       Louisville         79866.666667   80833.333333   81466.666667   
Texas          Lubbock                     NaN            NaN            NaN   
North Carolina Durham            129300.000000  132533.333333  135100.000000   
Florida        Orlando            88666.666667   90366.666667   92433.333333   
North Carolina Greensboro        104933.333333  106900.000000  108100.000000   
Nebraska       Lincoln           105966.666667  107600.000000  109700.000000   
Virginia       Chesapeake        121200.000000  122733.333333  123300.000000   
Wisconsin      Madison           137700.000000  140500.000000  141933.333333   
Tennessee      Knoxville          79666.666667   80166.666667   80466.666667   
New York       Rochester          53566.666667   53233.333333   53200.000000   
North Carolina Winston-Salem     100233.333333  101566.666667  102533.333333   
California     Irvine            326966.666667  336966.666667  341733.333333   
...                                        ...            ...            ...   
Pennsylvania   Edinboro           93900.000000   98166.666667   94366.666667   
               Collegeville      179966.666667  180100.000000  185000.000000   
Massachusetts  Paxton            173333.333333  177633.333333  182133.333333   
Pennsylvania   Kutztown          106966.666667  107966.666667  111766.666667   
Indiana        Upland             86700.000000   85900.000000   86400.000000   
Vermont        Castleton                   NaN            NaN            NaN   
Pennsylvania   Annville           87066.666667   87033.333333   86200.000000   
Kentucky       Wilmore           101266.666667  100266.666667  102000.000000   
New York       Clinton           164200.000000  169100.000000  176100.000000   
               Clinton            94700.000000   99633.333333  104433.333333   
Tennessee      Henderson          64933.333333   64000.000000   62066.666667   
Michigan       Olivet             85833.333333   90600.000000   91900.000000   
Indiana        Hanover            78566.666667   80866.666667   82033.333333   
Georgia        Waleska           123633.333333  125433.333333  128300.000000   
Michigan       Spring Arbor      118433.333333  122633.333333  125533.333333   
New Hampshire  Rindge            121866.666667  127300.000000  128233.333333   
North Carolina Elon              142100.000000  143033.333333  145066.666667   
Ohio           Hiram             154300.

In [50]:
non_uni_list = list(set(hdata.index.tolist()) - set(uni_town_data.index.tolist()))
non_uni_town_data = hdata.loc[non_uni_list]

In [51]:
non_uni_town_data

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3,ratio
State,RegionName,,,,,,,,,,,,,,,,,,,,,
Florida,Kendall West,85433.333333,86733.333333,88966.666667,90466.666667,91666.666667,94400.000000,98300.000000,101566.666667,105833.333333,110100.000000,...,151100.000000,157366.666667,160900.000000,165033.333333,167000.000000,171033.333333,174666.666667,180366.666667,184950.0,1.503414
Georgia,Moreland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,154800.000000,158700.000000,158133.333333,159266.666667,162700.000000,165100.000000,163700.000000,167000.000000,169400.0,NaN
Kentucky,Fisherville,209533.333333,214433.333333,214733.333333,215666.666667,217700.000000,220966.666667,219366.666667,220000.000000,221300.000000,221566.666667,...,279866.666667,284166.666667,292533.333333,295233.333333,295000.000000,294566.666667,289400.000000,289233.333333,301600.0,1.004048
Tennessee,Selmer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,58000.000000,59700.000000,60433.333333,62166.666667,63766.666667,66300.000000,66133.333333,68333.333333,70650.0,NaN
Indiana,Battle Ground,NaN,NaN,NaN,NaN,NaN,NaN,122100.000000,121366.666667,123733.333333,120633.333333,...,135600.000000,142200.000000,144866.666667,149700.000000,151333.333333,148500.000000,144166.666667,140466.666667,139400.0,0.887380
California,Banning,97733.333333,99033.333333,101500.000000,104966.666667,110766.666667,116933.333333,118233.333333,121000.000000,122600.000000,125700.000000,...,181833.333333,185966.666667,186500.000000,190400.000000,197500.000000,201966.666667,205400.000000,210966.666667,215150.0,1.371429
Florida,Southwest Ranches,283000.000000,293300.000000,306633.333333,317033.333333,325366.666667,337166.666667,343666.666667,352833.333333,359300.000000,372233.333333,...,532366.666667,548700.000000,553166.666667,565166.666667,579700.000000,586366.666667,582033.333333,588900.000000,598350.0,1.248284
New Jersey,Country Lake Estates,92866.666667,93800.000000,94266.666667,95600.000000,98100.000000,99600.000000,102933.333333,105200.000000,107333.333333,108700.000000,...,152166.666667,149566.666667,148600.000000,151566.666667,156833.333333,157333.333333,156400.000000,156633.333333,156850.0,1.036560
Arkansas,Ward,74900.000000,75133.333333,75266.666667,76166.666667,78100.000000,80000.000000,81100.000000,81500.000000,82366.666667,82066.666667,...,109033.333333,110000.000000,110166.666667,110333.333333,107966.666667,108333.333333,108366.666667,109466.666667,110750.0,1.004712


In [52]:
t,p = ttest_ind(uni_town_data['ratio'].dropna(),non_uni_town_data['ratio'].dropna())

In [53]:
t, p

(-2.998032664179183, 0.0027240637047611684)

In [54]:
different = True if p < 0.01 else False

In [55]:
better = "non-university town" if uni_town_data['ratio'].mean() > non_uni_town_data['ratio'].mean() else "university town"

In [56]:
better

'university town'

In [57]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return (different, p, better)

In [58]:
run_ttest()

(True, 0.0027240637047611684, 'university town')